# Web scraping using scrapy (xpath)

In [1]:
#import required libraries
from scrapy import Selector
import requests
import re
import pandas as pd

In [2]:
#url of web site
url="https://editorial.rottentomatoes.com/guide/essential-movies-to-watch-now/"

In [3]:
#get the content of response
r = requests.get(url).content

In [4]:
#create selector
sel = Selector(text = r)

## Extract movie URL

In [5]:
#test to extract first movie
sel.xpath('//*[@id="row-index-1"]/div[3]/div[1]/div[1]/div/div/h2/a').extract_first()

'<a href="https://www.rottentomatoes.com/m/1000013_12_angry_men">12 Angry Men</a>'

In [6]:
#get first movie url
first_movie= sel.xpath('//*[@id="row-index-1"]/div[3]/div[1]/div[1]/div/div/h2/a/@href').extract()[0]

print(first_movie)

https://www.rottentomatoes.com/m/1000013_12_angry_men


Extract all movies url 

In [7]:
#get movies url of first page
movies_url= sel.xpath('*//div[contains(@id,"row-index-")]/div[3]/div[1]/div[1]/div/div/h2/a/@href').extract()

In [8]:
#check number of movies
len(movies_url)

150

In [9]:
#show the list
movies_url

['https://www.rottentomatoes.com/m/1000013_12_angry_men',
 'https://www.rottentomatoes.com/m/2001_a_space_odyssey',
 'https://www.rottentomatoes.com/m/400_blows',
 'https://www.rottentomatoes.com/m/adventures_of_priscilla_queen_of_the_desert',
 'https://www.rottentomatoes.com/m/1000355-adventures_of_robin_hood',
 'https://www.rottentomatoes.com/m/aguirre_the_wrath_of_god',
 'https://www.rottentomatoes.com/m/airplane',
 'https://www.rottentomatoes.com/m/akira',
 'https://www.rottentomatoes.com/m/alien',
 'https://www.rottentomatoes.com/m/1000617-aliens',
 'https://www.rottentomatoes.com/m/1000626-all_about_eve',
 'https://www.rottentomatoes.com/m/all_about_my_mother',
 'https://www.rottentomatoes.com/m/all_the_presidents_men',
 'https://www.rottentomatoes.com/m/almost_famous',
 'https://www.rottentomatoes.com/m/amadeus',
 'https://www.rottentomatoes.com/m/amelie',
 'https://www.rottentomatoes.com/m/amour_2013',
 'https://www.rottentomatoes.com/m/american_in_paris',
 'https://www.rottent

Apply for loop to get all 300 movies

In [10]:
pages=[1,2]

In [11]:
#get all movies url
movie_url_list=[]
for page in range(len(pages)):
    r=requests.get(url+str(page+1)+'/').content
    sel = Selector(text = r)
    movie_url= sel.xpath('*//div[contains(@id,"row-index-")]/div[3]/div[1]/div[1]/div/div/h2/a/@href').extract()
    for movie in movie_url:
        movie_url_list.append(movie)

In [12]:
#check
len(movie_url_list)

300

In [13]:
#check
movie_url_list[0]

'https://www.rottentomatoes.com/m/1000013_12_angry_men'

## Test cells to get movie data

Watch the steps from [here](https://www.youtube.com/watch?v=NGaLVmLaVnM)

In [ ]:
#we use the cell to do intial test 
top=[]
r = requests.get(movie_url_list[1]).content
sel = Selector(text=r)
movie_name= sel.xpath('//*[@id="topSection"]/div[1]/score-board/h1/text()').extract()[0]
#audience_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board//div/div[2]/div[2]/div/score-icon-audience//div/span[2]').extract()
audience_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board/@audiencescore').extract()[0]
audience_rating = sel.xpath('//*[@id="topSection"]/div[1]/score-board/a[2]/text()').extract()[0].strip()[:-9].replace(',','')
#rt_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board//div/div[2]/div[1]/div/score-icon-critic//div/span[2]')
rt_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board/@tomatometerscore').extract()[0]
rt_reviwers=sel.xpath('//*[@id="topSection"]/div[1]/score-board/a[1]/text()').extract()[0].strip()[:-8].replace(',','')
genre= sel.xpath('//*[@id="topSection"]/div[1]/score-board/p/text()').extract()[0].split(',')[1]
release = sel.xpath('//*[@id="topSection"]/div[1]/score-board/p/text()').extract()[0].split(',')[0]
duration = sel.xpath('//*[@id="topSection"]/div[1]/score-board/p/text()').extract()[0].split(',')[2]

#convert duration from text to minutes
time= duration.split()
x=1
if len(time) == x :
    time = int(re.findall(r'\d+', time[0])[0])*60
else:
    time=int(re.findall(r'\d+', time[0])[0])*60 + int (re.findall(r'\d+', time[1])[0])

movie_info=sel.xpath('//*[@id="movieSynopsis"]/text()').extract()[0].strip()
#language= sel.xpath('//*[@id="mainColumn"]//section[6]/div/div/ul/li[2]/div[2]/text()').extract()[0].strip()
language= sel.xpath('//ul[@class="content-meta info"]/li[2]/div[2]/text()').extract()[0].strip()
#director= sel.xpath('//ul[@class="content-meta info"]/li[3]/div[2]/a/text()').extract()[0]
director= retrive(sel,'Director:')
#producer= sel.xpath('//ul[@class="content-meta info"]/li[4]/div[2]/a//text()').extract()
producer= retrive(sel,'Producer:')
print(producer)
writer=retrive(sel,'Writer:')
print(writer)
#
# if len(producer) > 1:
#     producer = producer[0] + ', ' + producer[1]
# else:
#     producer=producer[0]
    
# writer= sel.xpath('//ul[@class="content-meta info"]/li[5]/div[2]/a//text()').extract()
# if len(writer) > 1:
#     writer = writer[0] + ', ' + witer[1]
# else:
#     writer=writer[0]
actor_1 = sel.xpath('//*[@id="movie-cast"]/div/div/div[1]/div[2]/a/span/text()').extract()[0].strip()
actor_2 = sel.xpath('//*[@id="movie-cast"]/div/div/div[2]/div[2]/a/span/text()').extract()[0].strip()
actor_3 = sel.xpath('//*[@id="movie-cast"]/div/div/div[3]/div[2]/a/span/text()').extract()[0].strip()
actor_4 = sel.xpath('//*[@id="movie-cast"]/div/div/div[4]/div[2]/a/span/text()').extract()[0].strip()

print(producer)
top.append({
                     'name': movie_name,
                    'critics_score' : int(rt_score),
                    'critics_review_numbers' : int(rt_reviwers),
                    'audience_score' : int(audience_score),
                    'audience_reviews' :(audience_rating),
                    'year' : int(release),
                    'genre' : genre,
                    'duration' : duration,
                    'time_in_minutes' : time,
                    'movie_page' : movie_url,
                    'language': language,
                    'director': director,
                    'producer':producer,
                    'writer': writer,
                    'actor_1':actor_1,
                    'actor_2': actor_2,
                    'actor_3': actor_3,
                    'actor_4': actor_4})
df=pd.DataFrame(top, columns=['rank','name','critics_score','critics_review_numbers','audience_score', 'audience_reviews','year',
                    'genre', 'duration','time_in_minutes', 'movie_page', 'language', 'director','producer', 'writer', 'actor_1',
                    'actor_2', 'actor_3', 'actor_4'])
df.head()

In [34]:
sel.xpath('//ul[@class="content-meta info"]/li')

[<Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' data='<li class="meta-row clearfix" data-qa...'>,
 <Selector xpath='//ul[@class="content-meta info"]/li' 

In [18]:
sel.xpath('//ul[@class="content-meta info"]/li')[0].extract()

'<li class="meta-row clearfix" data-qa="movie-info-item">\n                    <div class="meta-label subtle" data-qa="movie-info-item-label">Rating:</div>\n                    <div class="meta-value" data-qa="movie-info-item-value">G\n                    </div>\n                </li>'

In [274]:
for li in sel.xpath('//ul[@class="content-meta info"]/li'):
    if li.xpath('//div[@class="meta-label subtle"]/text()').extract()[0].strip() == "Rating:":
        rating= li.xpath('//div[@class="meta-value"]/text()').extract()[0].strip()

In [275]:
print(rating)

R (Language|Brutal Disturbing Images|Strong Violence)


In [276]:
sel.xpath('//ul[@class="content-meta info"]/li').xpath('div[@class="meta-label subtle"]//text()').extract()

['Rating:',
 'Genre:',
 'Original Language:',
 'Director:',
 'Producer:',
 'Writer:',
 'Release Date (Theaters):',
 'Release Date (Streaming):',
 'Box Office (Gross USA):',
 'Runtime:',
 'Distributor:',
 'Sound Mix:']

In [189]:
sel.xpath('//ul[@class="content-meta info"]/li')[2].xpath('div[@class="meta-value"]//text()').extract()

['\n                        \n                            \n                                ',
 'Sidney Lumet',
 '\n                            \n                        \n                    ']

In [190]:
print([e.strip().replace('\n', '') for e in sel.xpath('//ul[@class="content-meta info"]/li')[2].xpath('div[@class="meta-value"]//text()').extract()])


['', 'Sidney Lumet', '']


In [203]:
xx=[e.strip().replace('\n', '') for e in sel.xpath('//ul[@class="content-meta info"]/li')[5].xpath('div[@class="meta-value"]//text()').extract() if e !=""]


In [204]:
xx

['', 'Arthur C. Clarke', ',', 'Stanley Kubrick', '']

In [205]:
[e  for e in xx if e not in ["", ',']][0] + ', ' + [e  for e in xx if e not in ["", ',']][1]

'Arthur C. Clarke, Stanley Kubrick'

In [257]:
for sel_elem in sel.xpath('//ul[@class="content-meta info"]/li'):
    if sel_elem.xpath('div[@class="meta-label subtle"]//text()').extract()[0] == 'Rating:':
       
        list_of_text= [e.strip().replace('\n', '') for e in sel_elem.xpath('div[@class="meta-value"]//text()').extract() if e !=""]
        #print(list_of_text)
        #print(',' in list_of_text)
        if ',' in list_of_text:
            value =[e  for e in list_of_text if e not in ["", ',']][0] + ', ' + [e  for e in list_of_text if e not in ["", ',']][1]
            print(value)
        else:
            value = [e  for e in list_of_text if e not in ["", ',']][0]
            print(value)

G


In [261]:
#test
retrive(sel, 'Writer:')

'Arthur C. Clarke, Stanley Kubrick'

## Don't Repeat Your Self

Create function to get below data

- Rating
- language
- director
- producer
- writer

In [258]:

def retrive(sel, key):
    """ function to retive 
        Rating, Language, Director, Producer and writer
        
        Paramters:
            sel: selector of movie page
            key: key to search for needed data
            
        Returns:
            the value of the key
    """
    for sel_elem in sel.xpath('//ul[@class="content-meta info"]/li'):
        if sel_elem.xpath('div[@class="meta-label subtle"]//text()').extract()[0] == key:
            list_of_text= [e.strip().replace('\n', '') for e in sel_elem.xpath('div[@class="meta-value"]//text()').extract() if e !=""]
            if ',' in list_of_text:
                value =[e  for e in list_of_text if e not in ["", ',']][0] + ', ' + [e  for e in list_of_text if e not in ["", ',']][1]
                #print(value)
            else:
                value = [e  for e in list_of_text if e not in ["", ',']][0]
                #print(value)
            return value

## Loop over all pages after solving all problem in test phase
- If you face any error get the page that cause error 
- Start to test why error happened in test cell
- Add the needed steps to overcome the error and get needed data below

In [268]:
top_300=[]
for movie_url in movie_url_list:
    r = requests.get(movie_url).content
    sel = Selector(text=r)
    movie_name= sel.xpath('//*[@id="topSection"]/div[1]/score-board/h1/text()').extract()[0]
    #audience_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board//div/div[2]/div[2]/div/score-icon-audience//div/span[2]').extract()
    audience_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board/@audiencescore').extract()[0]
    audience_rating = sel.xpath('//*[@id="topSection"]/div[1]/score-board/a[2]/text()').extract()[0].strip()[:-9].replace(',','')
    #rt_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board//div/div[2]/div[1]/div/score-icon-critic//div/span[2]')
    rt_score = sel.xpath('//*[@id="topSection"]/div[1]/score-board/@tomatometerscore').extract()[0]
    rt_reviwers=sel.xpath('//*[@id="topSection"]/div[1]/score-board/a[1]/text()').extract()[0].strip()[:-8].replace(',','')
    genre= sel.xpath('//*[@id="topSection"]/div[1]/score-board/p/text()').extract()[0].split(',')[1]
    release = sel.xpath('//*[@id="topSection"]/div[1]/score-board/p/text()').extract()[0].split(',')[0]
    duration = sel.xpath('//*[@id="topSection"]/div[1]/score-board/p/text()').extract()[0].split(',')[2]

    #convert duration from text to minutes
    time= duration.split()
    x=1
    if len(time) == x :
        time = int(re.findall(r'\d+', time[0])[0])*60
    else:
        time=int(re.findall(r'\d+', time[0])[0])*60 + int (re.findall(r'\d+', time[1])[0])

    movie_info=sel.xpath('//*[@id="movieSynopsis"]/text()').extract()[0].strip()
    #language= sel.xpath('//*[@id="mainColumn"]//section[6]/div/div/ul/li[2]/div[2]/text()').extract()[0].strip()
    #language= sel.xpath('//ul[@class="content-meta info"]/li[2]/div[2]/text()').extract()[0].strip()
    
    Rating= retrive(sel, 'Rating:')
    language = retrive(sel, 'Original Language:')
    director= retrive(sel,'Director:')
    producer= retrive(sel,'Producer:')
    writer=retrive(sel,'Writer:')
    
    actor_1 = sel.xpath('//*[@id="movie-cast"]/div/div/div[1]/div[2]/a/span/text()').extract()[0].strip()
    actor_2 = sel.xpath('//*[@id="movie-cast"]/div/div/div[2]/div[2]/a/span/text()').extract()[0].strip()
    actor_3 = sel.xpath('//*[@id="movie-cast"]/div/div/div[3]/div[2]/a/span/text()').extract()[0].strip()
    actor_4 = sel.xpath('//*[@id="movie-cast"]/div/div/div[4]/div[2]/a/span/text()').extract()[0].strip()
    top_300.append({
                     'name': movie_name,
                    'critics_score' : int(rt_score),
                    'critics_review_numbers' : int(rt_reviwers),
                    'audience_score' : int(audience_score),
                    'audience_reviews' :(audience_rating),
                    'year' : int(release),
                    'genre' : genre,
                    'duration' : duration,
                    'time_in_minutes' : time,
                    'movie_page' : movie_url,
                    'language': language,
                    'producer':producer,
                    'writer': writer,
                    'actor_1':actor_1,
                    'actor_2': actor_2,
                    'actor_3': actor_3,
                    'actor_4': actor_4})
df=pd.DataFrame(top_300, columns=['name','critics_score','critics_review_numbers','audience_score', 'audience_reviews','year',
                    'genre', 'duration','time_in_minutes', 'movie_page', 'language', 'producer', 'writer', 'actor_1',
                    'actor_2', 'actor_3', 'actor_4'])
df.head()

,rank,name,critics_score,critics_review_numbers,audience_score,audience_reviews,year,genre,duration,time_in_minutes,movie_page,language,producer,writer,actor_1,actor_2,actor_3,actor_4
0,NaN,12 Angry Men,100,60,97,100000,1957,Drama,1h 35m,95,https://www.rottentomatoes.com/m/1000013_12_an...,English,"Henry Fonda, Reginald Rose",Reginald Rose,Henry Fonda,Lee J. Cobb,Ed Begley,E.G. Marshall
1,NaN,2001: A Space Odyssey,91,117,89,250000,1968,Sci-fi,2h 19m,139,https://www.rottentomatoes.com/m/2001_a_space_...,English,Stanley Kubrick,"Arthur C. Clarke, Stanley Kubrick",Keir Dullea,Gary Lockwood,William Sylvester,Daniel Richter
2,NaN,The 400 Blows,99,71,94,25000,1959,Crime/Drama,1h 33m,93,https://www.rottentomatoes.com/m/400_blows,French (France),François Truffaut,"François Truffaut, François Truffaut",Jean-Pierre Léaud,Claire Maurier,Albert Remy,Guy Decomble
3,NaN,"The Adventures of Priscilla, Queen of the Desert",94,48,88,25000,1994,Comedy/Lgbtq+,1h 42m,102,https://www.rottentomatoes.com/m/adventures_of...,English,"Al Clark, Michael Hamlyn",Stephan Elliott,Terence Stamp,Hugo Weaving,Guy Pearce,Bill Hunter
4,NaN,The Adventures of Robin Hood,100,50,89,25000,1938,Adventure/Action,1h 42m,102,https://www.rottentomatoes.com/m/1000355-adven...,English,Hal B. Wallis,"Norman Reilly Raine, Seton I. Miller",Errol Flynn,Olivia de Havilland,Basil Rathbone,Claude Rains


In [271]:
df.tail(10)

,rank,name,critics_score,critics_review_numbers,audience_score,audience_reviews,year,genre,duration,time_in_minutes,movie_page,language,producer,writer,actor_1,actor_2,actor_3,actor_4
290,NaN,Wendy and Lucy,86,187,67,10000,2008,Drama,1h 20m,80,https://www.rottentomatoes.com/m/wendy_and_lucy,English,None,"Jonathan Raymond, Jonathan Raymond",Michelle Williams,Will Patton,Will Oldham,John Robinson
291,NaN,West Side Story,92,115,84,100000,1961,Musical,2h 31m,151,https://www.rottentomatoes.com/m/west_side_story,English,Robert Wise,"Jerome Robbins, Arthur Laurents",Natalie Wood,Richard Beymer,Russ Tamblyn,George Chakiris
292,NaN,What's Love Got to Do With It,96,57,88,50000,1993,Biography,1h 58m,118,https://www.rottentomatoes.com/m/whats_love_go...,English,"Doug Chapin, Barry Krost","Tina Turner, Kurt Loder",Angela Bassett,Laurence Fishburne,Vanessa Bell Calloway,Jenifer Lewis
293,NaN,The Wild Bunch,91,65,90,25000,1969,Western,2h 22m,142,https://www.rottentomatoes.com/m/1059489-wild_...,English,Phil Feldman,"Walon Green, Roy N. Sickner",William Holden,Ernest Borgnine,Robert Ryan,Edmond O'Brien
294,NaN,Wild Strawberries,94,48,94,10000,1957,Drama,1h 30m,90,https://www.rottentomatoes.com/m/wild_strawber...,Swedish,Allan Ekelund,Ingmar Bergman,Victor Sjöström,Ingrid Thulin,Bibi Andersson,Gunnar Björnstrand
295,NaN,The Wizard of Oz,98,166,89,250000,1939,Kids & family/Fantasy,1h 41m,101,https://www.rottentomatoes.com/m/the_wizard_of...,English,"Victor Fleming, Mervyn LeRoy","L. Frank Baum, Noel Langley",Judy Garland,Frank Morgan,Ray Bolger,Bert Lahr
296,NaN,Wonder Woman,93,479,83,100000,2017,Action/Adventure,2h 21m,141,https://www.rottentomatoes.com/m/wonder_woman_...,English,"Charles Roven, Deborah Snyder",Allan Heinberg,Gal Gadot,Chris Pine,Connie Nielsen,Robin Wright
297,NaN,Y tu mamá también,92,138,88,50000,2001,Comedy/Drama,1h 45m,105,https://www.rottentomatoes.com/m/y_tu_mama_tam...,Spanish,"Alfonso Cuarón, Jorge Vergara","Alfonso Cuarón, Carlos Cuarón",Maribel Verdú,Gael García Bernal,Diego Luna,Diana Bracho
298,NaN,Your Sister's Sister,83,145,66,10000,2011,Comedy/Drama,1h 30m,90,https://www.rottentomatoes.com/m/your_sisters_...,English,Steven Schardt,Lynn Shelton,Emily Blunt,Rosemarie DeWitt,Mark Duplass,Mike Birbiglia
299,NaN,Zero Dark Thirty,91,309,80,100000,2012,Drama/Mystery & thriller,2h 37m,157,https://www.rottentomatoes.com/m/zero_dark_thirty,English,"Mark Boal, Kathryn Bigelow",Mark Boal,Jessica Chastain,Jason Clarke,Joel Edgerton,Jennifer Ehle


In [272]:
len(top_300)

300